Train a multiple output network
---

GO

In [42]:
from __future__ import print_function
import pandas as pd
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

***Training parameters***


In [43]:
# data property
min_phi = 0
max_phi = 360
min_eta = -10
max_eta = 10
min_delta = -10
max_delta = 10
num_phi_classes = max_phi - min_phi
num_eta_classes = max_eta - min_eta
num_delta_classes = max_delta - min_delta

image_size_x = 200
image_size_y = 200
color_channel = 1

batch_size = 32
epochs = 20
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

n = 1
depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%d' % (depth)


In [44]:
# Input image dimensions.
# input_shape = (train_generator.batch_size, image_size_x, image_size_y, 1)
input_shape = (image_size_x, image_size_y, 1)

***Learning Rate Schedule***

In [45]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


***2D Convolution-Batch Normalization-Activation stack builder***

In [46]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


***ResNet Version 2 Model builder ***

In [47]:
def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add 3 classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    
    # phi
    x_phi = Activation('relu')(x)
    x_phi = AveragePooling2D(pool_size=8)(x_phi)
    y_phi = Flatten()(x_phi)
    phi_output = Dense(1,
                    kernel_initializer='normal')(y_phi)
    
    # eta
    x_eta = Activation('relu')(x)
    x_eta = AveragePooling2D(pool_size=8)(x_eta)
    y_eta = Flatten()(x_eta)
    eta_output = Dense(1,
                    kernel_initializer='normal')(y_eta)
    
    # delta
    x_delta = Activation('relu')(x)
    x_delta = AveragePooling2D(pool_size=8)(x_delta)
    y_delta = Flatten()(x_delta)
    delta_output = Dense(1,
                    kernel_initializer='normal')(y_delta)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=[phi_output, eta_output, delta_output])
    return model

***Compile model***

In [48]:
model = resnet_v2(input_shape=input_shape, depth=depth)

In [49]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['mae','accuracy'])
model.summary()
print(model_type)

Learning rate:  0.001
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 200, 200, 16) 160         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_31 (BatchNo (None, 200, 200, 16) 64          conv2d_40[0][0]                  
__________________________________________________________________________________________________
activation_37 (Activation)      (None, 200, 200, 16) 0           batch_normalization_31[0][0]     
______________________________________________________________________

***Prepare model model saving directory.***

In [50]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'alice_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

***Prepare callbacks for model saving and for learning rate adjustment.***

In [51]:
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

***Run training, with data augmentation.***

In [52]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=1./255,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.2)

Using real-time data augmentation.


***Read data***

In [53]:
image_and_labels=pd.read_csv('data_and_labels.csv')
image_and_labels
train_generator=datagen.flow_from_dataframe(
    dataframe=image_and_labels, 
    x_col='images', 
    y_col=['phi', 'eta', 'delta'], 
    class_mode='multi_output', 
    color_mode='grayscale', 
    target_size=(image_size_x, image_size_y), 
    batch_size=batch_size
)

Found 28720 validated image filenames.


***Fit the model on the batches generated by datagen.flow_from_dataframe().***

In [ ]:
model.fit_generator(train_generator,
                    epochs=epochs, 
                    verbose=1, 
                    workers=4,
                    callbacks=callbacks)

Epoch 1/20
Learning rate:  0.001
800/898 [=========================>....] - ETA: 1:24 - loss: 0.7074 - dense_10_loss: 0.1217 - dense_11_loss: 0.1498 - dense_12_loss: 0.1733 - dense_10_mae: 0.2729 - dense_10_accuracy: 0.0000e+00 - dense_11_mae: 0.2918 - dense_11_accuracy: 0.0000e+00 - dense_12_mae: 0.3035 - dense_12_accuracy: 0.0000e+00